In [ ]:
from workingmem.utils import get_wandb_runs
from pathlib import Path
import pandas as pd

In [ ]:
project_name = "wm-comp-limit-7.3.1"
# stores td_prob, role_n_congruence, n_back
sweep_data = {
    "6wp68dnn": (0, 0, 5),
    "1scqx6ya": (0, 0.5, 5),
    "pfq1xma0": (0, 1, 5),
    "hcv8c8lu": (0.5, 0, 5),
    "u4rf4nbw": (0.5, 0.5, 5),
    "cxuud4eg": (0.5, 1, 5),
    "a1ulm0zh": (1, 0, 5),
    "oqclfmfe": (1, 0.5, 5),
    "4y7je0eo": (1, 1, 5),
}


In [ ]:
for sweep_id in sweep_data:
    try:
        td_prob, role_n_congruence, n_back = sweep_data[sweep_id]
        fname = f"{sweep_id}_tdprob={td_prob}_rolencong={role_n_congruence}_nback={n_back}_roles=50_concurrent=4.csv"
        print(f"fetching for: {fname}")

        df = get_wandb_runs(project_name, sweep_id)
        df["td_prob"], df["role_n_congruence"], df["n_back"] = (
            td_prob,
            role_n_congruence,
            n_back,
        )

        print(f"saving: {fname}")
        df.to_csv(fname)
    except KeyError as e:
        print(e, sweep_id, sweep_data[sweep_id])